## Monopoly Game Simulator

The following notebook runs a simulation of the game Monopoly with multiple agents acting as fast mind and slow mind. 

It is built [on top of this Monopoly simulator](https://github.com/giogix2/MonopolySimulator)

### Project Setup

Install dependencies

In [121]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Import dependencies

In [122]:
# monopoly simulator
from simulator.monosim.player import Player
from simulator.monosim.board import get_board, get_roads, get_properties, get_community_chest_cards, get_bank

# LangChain
from openai import OpenAI
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain import LLMChain, PromptTemplate

# General dependencies
import os
import csv
import sys
import json
import time
import shutil
import random
import logging
from functools import wraps
from datetime import datetime
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Union, Dict, Any, Optional

In [123]:
# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

In [124]:
# Logging utilities
class StructuredLogger:
    """
    A logger that outputs to TSV files for structured logging of LLM interactions.
    """

    # Define all possible message types for documentation
    MESSAGE_TYPES = [
        "vectorstore_add",           # Adding a case to vectorstore
        "vectorstore_retrieve",      # Retrieving similar cases
        "vectorstore_summarize",     # Summarizing cached decisions
        "pro_argument_prompt",       # Input prompt for pro arguments
        "pro_argument_response",     # Response from pro argument LLM
        "con_argument_prompt",       # Input prompt for con arguments
        "con_argument_response",     # Response from con argument LLM
        "judge_prompt",              # Input prompt for judge
        "judge_response",            # Response from judge LLM
        "fast_prompt",               # Input prompt for fast LLM
        "fast_response",             # Fast LLM respnose
        "slow_prompt",               # Input prompt for slow LLM
        "slow_response",             # Slow LLM response

        # Monopoly-specific messages
        "property_details",          # Property information being considered
        "player_stats",              # Player statistics at decision time
        "default_decision",          # Default player's decision
        "step_num",                  # Step number in game
        "game_init",                 # Game initialization config
        "player_summary"             # Players' state at decision time
    ]

    def __init__(self, tsv_filepath: str):
        """
        Initialize the logger with a base filepath.

        Args:
            tsv_filepath: Filepath for TSV log files (includes timestamp)
        """
        self.tsv_filepath = tsv_filepath

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(self.tsv_filepath), exist_ok=True)

        # Initialize TSV file with headers if it doesn't exist
        if not os.path.exists(self.tsv_filepath):
            with open(self.tsv_filepath, "w", newline="", encoding="utf-8") as tsvfile:
                writer = csv.writer(tsvfile, delimiter="\t")
                writer.writerow(["timestamp", "message_type", "message"])

    def log(self, message_type: str, message_data: Union[str, Dict[str, Any], list[Any], BaseModel]) -> None:
        """
        Log a message to the TSV file.

        Args:
            message_type: Type of message (one of MESSAGE_TYPES)
            message_data: Data to be logged (will be converted to JSON if not already a string)
        """
        if message_type not in self.MESSAGE_TYPES:
            raise ValueError(
                f"Invalid message_type: {message_type}. Must be one of {self.MESSAGE_TYPES}"
            )

        # Convert message_data to string if it's not already
        if isinstance(message_data, BaseModel):
            message = json.dumps(message_data.model_dump(), ensure_ascii=False)
        elif isinstance(message_data, (dict, list)):
            message = json.dumps(message_data, ensure_ascii=False)
        else:
            message = str(message_data)

        timestamp = datetime.now().isoformat()

        # Append to TSV file
        with open(self.tsv_filepath, "a", newline="", encoding="utf-8") as tsvfile:
            writer = csv.writer(tsvfile, delimiter="\t", quotechar='`', quoting=csv.QUOTE_MINIMAL)
            writer.writerow([timestamp, message_type, message])

### Fast Mind Slow Mind Architecture

Setting up output parsers and prompt templates.

In [125]:
template = PromptTemplate(
    input_variables=["property_details", "player_stats", "related_decisions"],
    template="""
    You are a strategic decision-maker playing a game of Monopoly.

    Rules of the game:
    1. Buying: Players can buy unowned properties they land on for the listed price. Mortgaging or trading properties owned properties is NOT allowed.
    2. Building: Rent at a property increases with houses or hotels built. Four houses can be upgraded to a hotel.
    3. Rent Bonus: Rent increases if a player owns all three properties of a color.
    4. Winning: The last player remaining after others go bankrupt wins.

    Your goal:
    Decide whether to buy a property in Monopoly. Maximize your chances of winning the game with forward-thinking reasoning.

    Past Related Decisions:
    {related_decisions}

    Current Property Details:
    {property_details}

    Current Player Details:
    {player_stats}

    Should the player buy this property? Justify your reasoning briefly.
    Then, make your final decision. Output True to buy the property or False to not buy the property.
    Finally, output a decimal percentage uncertainty between 0-1 to indicate your uncertainty in the decision.
    """
)

In [126]:
class Output(BaseModel):
    reasoning: str = Field(description="Your reasoning for the decision")
    decision: bool = Field(description="The final decision (True for yes, False for no)")
    uncertainty: float = Field(description="Score between 0-1 on how uncertain you are about your vote. 0 if you are completely certain, 1 if you are completely uncertain")

### Reflection Vectorstore

In [127]:
def init_vectorstore(timestamp_str: str, game_index: int, 
                     vectorstore_path: str = "vectorstore/fastslow") -> FAISS:
    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

    if os.path.exists(vectorstore_path):
        # Create archive directory with timestamp and game index
        archive_path = f"{vectorstore_path}/archive/{timestamp_str}/{game_index}"
        os.makedirs(archive_path, exist_ok=True)
        
        # Move existing files to archive if they exist
        for file in ['index.faiss', 'index.pkl']:
            src = os.path.join(vectorstore_path, file)
            if os.path.exists(src):
                dst = os.path.join(archive_path, file)
                shutil.move(src, dst)

    dummy_doc = [Document(page_content="Initialized")]
    vectorstore = FAISS.from_documents(dummy_doc, embedding_model)  
    vectorstore.save_local(vectorstore_path)  
    return vectorstore


def retrieve_similar(query: str, vectorstore: FAISS, k: int = 2, logger: Optional[StructuredLogger] = None):
    """
    Retrieve similar documents.
    """
    results = vectorstore.similarity_search(query, k=k)
    logging.info("===")
    logging.info(f"Query: {query}")
    logging.info(f"Retrieved similar documents: {json.dumps([doc.model_dump_json() for doc in results], indent=2)}")
    logging.info("===")
    
    # Log results to structured TSV
    if logger:
        serializable_results = [
            {"content": doc.page_content, "metadata": doc.metadata, "query": query}
            for doc in results
        ]
        logger.log("vectorstore_retrieve", serializable_results)
    
    return results if results else []

def summarize_decisions(results: List[Document], logger: Optional[StructuredLogger] = None) -> Document:
    """
    Summarize the decisions before adding to Vectorstore
    """
    template = PromptTemplate(
        input_variables=["decisions"],
        template="""
        You are a strategic decision-maker reviewing past decisions in Monopoly.

        Decisions:
        {decisions}

        Summarize the decisions and reflect on the most important implications for future strategic decisions.
        """
    )
    summary_chain = template | ChatOpenAI(model="gpt-4o")
    input_decisions = "\n".join([doc.page_content for doc in results])
    summary = summary_chain.invoke({"decisions": input_decisions}).content

    logging.info("===")
    logging.info(f"Input decisions: {input_decisions}")
    logging.info(f"Summarized decisions: {summary}")
    logging.info("===")

    if logger:
        logger.log("vectorstore_summarize", {
            "input_decisions": input_decisions, 
            "summary": summary
        })

    return Document(page_content=summary)

def add_to_vectorstore(case_id: str, 
                       context: str, 
                       arguments: str, 
                       decision: str, 
                       vectorstore: FAISS, 
                       CACHED_DECISIONS: list, 
                       reflection: bool=False, 
                       vectorstore_path: str = "vectorstore/fastslow", 
                       logger: Optional[StructuredLogger]=None) -> list:
    """
    Add a new document to the vectorstore for future retrieval
    """
    
    new_case = Document(
        page_content=f"Context: {context}\nArguments: {arguments}\nDecision: {decision}",
        metadata={"case_id": case_id, "timestamp": datetime.now().isoformat()}
    )
    logging.info("===")
    logging.info(f"Adding document: {json.dumps(new_case.model_dump_json(), indent=2)}")
    logging.info("===")

    # Log the operation if logger is provided
    if logger:
        logger.log("vectorstore_add", new_case)

    if reflection:
        CACHED_DECISIONS.append(new_case)
        if len(CACHED_DECISIONS) >= 3:
            summary = summarize_decisions(CACHED_DECISIONS)
            vectorstore.add_documents([summary])
            vectorstore.save_local(vectorstore_path)
            CACHED_DECISIONS.clear()
            # Log summary if logger provided
            if logger:
                logger.log("vectorstore_summarize", {"num_decisions": 5, "summary": summary.page_content})
    else:
        vectorstore.add_documents([new_case])
        vectorstore.save_local(vectorstore_path)  # Save updates locally

    return CACHED_DECISIONS

### Buying Properties
This function can be modified to get the agent to behave differently when deciding whether to buy a property. The placeholders are just examples of what you can do. Note: you don't need to use all the information available when making a decision.

In [128]:
# Langchain output parsing was not working, so using OpenAI's structured outputs
def get_openai_response(prompt: str, model: str, max_tokens: int, temperature: float, response_format: BaseModel):
    client = OpenAI()

    completion = client.beta.chat.completions.parse(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=[
            {"role": "user", "content": prompt}
        ],
        response_format=response_format,
    )

    return completion.choices[0].message.parsed

def safe_call(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        time.sleep(5)

        try:
            return func(*args, **kwargs)
        except Exception as e:
            logging.error(f"Error: {e}")
            print(f"Error: {e}")
            sys.exit(1)

In [129]:
def custom_buy(self, dict_property_info : dict) -> Union[str, None]:
    """
    Edit this function to determine whether the player should buy a property. 

    Return:
    YOUR FUNCTION MUST RETURN "buy" (type str) to buy the property 
    or None (type None) to not buy the property.
    
    WARNING: I've included exhaustive documentation on all the data you can 
    use to make this decision. DON'T use all of it; it's mostly irrelevant. 

    Parameters:
    self: useful for accessing the following methods
    - self.get_state(): returns player data in a dict with the keys
        - cash: the amount of cash the player has
        - owned_roads: a list of owned roads
        - owned_stations: a list of owned stations
        - owned_utilities: a list of owned utilities
        - owned_colors: a dict which maps colors to True if the player owns all 
            properties of that color. Ex: owned_colors['blue'] = True/False
        - owned_houses_hotels: a dict which maps property names to the number of 
            houses/hotels built there. Ex: owned_houses_hotels['Park Place'] = 3
        - mortgageable_amount: total mortgageable value of player's properties
        WARNING: PROBABLY NOT USEFUL DATA BELOW
        - morgaged_roads: a list of mortgaged roads
        - mortgaged_stations: a list of mortgaged stations
        - mortgaged_utilities: a list of mortgaged utilities
        - name: name of player
        - number: player number
        - position: player's position on the board
        - dice_value: the sum of the dice rolled
        - jail_count: the number of turns the player has been in jail
        - exit_jail: whether the player has exited jail
        - free_visit: whether the player is visiting jail for free
        - has_lost: whether the player has lost the game
    - self._dict_properties[property_name]: returns property info for an 
    arbitrary property with a dict like dict_property_info below

    dict_property_info: Has a dict with info about the property to buy/not buy. 
        The dict has the following keys:
    * name: the name of the property
    * belongs_to: the player who owns the property
    * price: the price of the property
    * rent: the rent of the property
    - color: the color of the property
    - rent_with_color_set: rent when all properties of same color are owned
    * type: the type of property (one of 'road', 'station', 'utility')
    - houses_cost: the cost of building a house
    - hotels_cost: the cost of building a hotel
    - rent_with_X_houses_0_hotels: rent when X = {1, 2, 3, 4} houses are built
    - rent_with_4_houses_1_hotels: rent when 4 houses and 1 hotel are built
    WARNING: PROBABLY NOT USEFUL DATA BELOW
    * mortgage_value: the amount of money earned when mortgaging the property
    * unmortgage_value: the amount of money cost to unmortgage the property
    * is_mortgaged: whether the property is mortgaged
    * board_num: the position of the property on the board
    P.S. (* means for all property types, - means for road properties only)
    """

    logging.info(f"Custom buy function called for property: {dict_property_info['name']}")

    # Get model names and parameters from environment variables
    primary_model = os.getenv("PRIMARY_MODEL_NAME", "gpt-4o-mini")
    secondary_model = os.getenv("SECONDARY_MODEL_NAME", "gpt-4o")
    reflect_before_vectorstore = os.getenv("REFLECT_BEFORE_VECTORSTORE", "False").lower() == "true"
    primary_params = os.getenv("PRIMARY_MODEL_PARAMS", None)
    secondary_params = os.getenv("SECONDARY_MODEL_PARAMS", None)
    if primary_params:
        primary_params = json.loads(primary_params)
    if secondary_params:
        secondary_params = json.loads(secondary_params)

    # Get the logger from environment if available
    logger_path = os.getenv("LOGGER_PATH")
    logger = None
    if logger_path:
        logger = StructuredLogger(logger_path)

    # currently, no trading. So don't buy property if someone else owns it
    if dict_property_info['belongs_to'] != None:
        return None

    ##################################################
    # Step 1: Gather game state and property data
    ##################################################
    property_details = f"""
    Property Name: {dict_property_info['name']}
    Type: {dict_property_info['type']}
    Cost: {dict_property_info['price']}
    Base Rent: {dict_property_info['rent']}
    """
    
    # Add specific details for each property type
    if dict_property_info['type'] == 'road':
        # Calculate number of properties of the same color owned
        n_color_properties = 0
        for property_name in self.get_state()['owned_roads']:
            property_name = property_name.strip().lower()  # normalize name
            prop_info = self._dict_properties.get(property_name)  # safe access
            if prop_info and prop_info['color'] == dict_property_info['color']:
                n_color_properties += 1

        property_details += f"""
        Color: {dict_property_info['color']}
        Cost of Building a House: {dict_property_info['houses_cost']}
        Cost of Building a Hotel: {dict_property_info['hotels_cost']}
        Rent with 4 Houses and 1 Hotel: {dict_property_info['rent_with_4_houses_1_hotels']}
        Number of {dict_property_info['color']} properties owned: {n_color_properties}
        """
    elif dict_property_info['type'] == 'station':
        property_details += f"""
        Number of Stations Owned: {len(self.get_state()['owned_stations'])}
        """
    elif dict_property_info['type'] == 'utility':
        property_details += f"""
        Number of Utilities Owned: {len(self.get_state()['owned_utilities'])}
        """

    player_stats = f"""
    Cash: {self.get_state()['cash']}
    Owned Roads: {len(self.get_state()['owned_roads'])}
    Owned Stations: {len(self.get_state()['owned_stations'])}
    Owned Utilities: {len(self.get_state()['owned_utilities'])}
    """

    past_cases = retrieve_similar(
        f"Property details:\n{property_details}", 
        self._vec, 
        k=2, 
        logger=logger
    )
    past_cases_text = "\n".join([case.page_content for case in past_cases])
    
    ##################################################
    # Step 2: LLM generation
    ##################################################
    UNCERTAINTY_THRESHOLD = 0.2

    fast_message = fast_message = template.format(
        property_details=property_details,
        player_stats=player_stats,
        related_decisions=past_cases_text
    )
    fast_response = safe_call(
        get_openai_response,
        prompt=fast_message,
        model=primary_model,
        max_tokens=primary_params.get("max_tokens", 4096),
        temperature=primary_params.get("temperature", 1),
        response_format=Output
    )
    decision = fast_response

    if fast_response.uncertainty >= UNCERTAINTY_THRESHOLD:
        slow_message = template.format(
            property_details=property_details,
            player_stats=player_stats,
            related_decisions=past_cases_text
        )
        slow_response = safe_call(
            get_openai_response,
            prompt=slow_message,
            model=secondary_model,
            max_tokens=secondary_params.get("max_tokens", 4096),
            temperature=secondary_params.get("temperature", 1),
            response_format=Output
        )
        decision = slow_response

    # ##################################################
    # # Step 3: Store the decision in the vectorstore and log
    # ##################################################
    logging.info("===")
    logging.info(f"Fast input: {fast_message}")
    logging.info(f"Fast reasoning: {fast_response.reasoning}")
    logging.info(f"Fast decision: {fast_response.decision}")
    logging.info(f"Fast uncertainty: {fast_response.uncertainty}")
    logging.info("===")
    if fast_response.uncertainty >= UNCERTAINTY_THRESHOLD:
        logging.info(f"Slow input: {slow_message}")
        logging.info(f"Slow reasoning: {slow_response.reasoning}")
        logging.info(f"Slow decision: {slow_response.decision}")
        logging.info(f"Slow uncertainty: {slow_response.uncertainty}")
        logging.info("===")

    if logger:
        logger.log("property_details", {
            "name": dict_property_info['name'],
            "type": dict_property_info['type'],
            "price": dict_property_info['price'],
            "rent": dict_property_info['rent'],
            "color": dict_property_info.get('color', None)
        })
        logger.log("player_stats", {
            "cash": self.get_state()['cash'],
            "owned_roads_count": len(self.get_state()['owned_roads']),
            "owned_stations_count": len(self.get_state()['owned_stations']),
            "owned_utilities_count": len(self.get_state()['owned_utilities']),
            "player_name": self.get_state()['name']
        })
        logger.log("fast_prompt", fast_message)
        logger.log("fast_response", fast_response)
        if fast_response.uncertainty >= UNCERTAINTY_THRESHOLD:
            logger.log("slow_prompt", slow_message)
            logger.log("slow_response", slow_response)

    cached_decisions = add_to_vectorstore(
        case_id=f"{dict_property_info['name']}_{self.get_state()['name']}",
        context=f"Property Details:\n{property_details}\nPlayer Stats:\n{player_stats}",
        arguments=f"Reasoning: {decision.reasoning}",
        decision=f"Decision: {"Buy" if decision.decision else "Don't buy"}\nUncertainty: {decision.uncertainty}",
        vectorstore=self._vec,
        CACHED_DECISIONS=self._cached_decisions,
        reflection=reflect_before_vectorstore,
        logger=logger
    )
    self._cached_decisions = cached_decisions
    

    # Return Decision
    return "buy" if decision.decision else None

### Backend Setup (Do not edit)

These are functions that setup a game and let you access the state of the game.

In [130]:
def log_method(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):

        result = func(self, *args, **kwargs)
        logging.info("===")
        logging.info(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        logging.info(f"{func.__name__} returned: {result}")
        logging.info("===")

        log_path = os.getenv("LOGGER_PATH")
        if log_path:
            logger = StructuredLogger(log_path)
            logger.log("default_decision", {
                "args": args,
                "kwargs": kwargs,
                "result": result
            })
        
        return result
    return wrapper

class LoggedPlayer(Player):
    @log_method
    def buy_or_bid(self, *args, **kwargs):
        return super().buy_or_bid(*args, **kwargs)

def modify_buy_or_bid(buy) -> str:
    return custom_buy

class CustomPlayer(Player):
    buy_or_bid = modify_buy_or_bid(Player.buy_or_bid)
    # Will be overwritten by real vectorstore in init_game function
    _vec = None
    _cached_decisions = []

In [131]:
def initialize_game(vec: FAISS) -> dict:
    """
    Initializes a game with two players and sets up the bank, board, roads, properties, 
    and community chest cards.
    
    Returns:
        dict: A dictionary containing the following:
            - "bank": Game's bank object.
            - "board": Main game board.
            - "roads": List of road objects.
            - "properties": List of property objects.
            - "community_chest_cards": Dictionary of community chest cards.
            - "players": List of two Player objects, with Player 1 first.
    """
    
    bank = get_bank()
    board = get_board()
    roads = get_roads()
    properties = get_properties()
    community_chest_cards = get_community_chest_cards()
    community_cards_deck = list(community_chest_cards.keys())

    # Note how we have one of our players vs. a default player that just buys 
    # whenever cash is available. We can change this later
    player1 = CustomPlayer('Alice', 1, bank, board, roads, properties, community_cards_deck)
    player1._vec = vec
    player2 = LoggedPlayer('Bob', 2, bank, board, roads, properties, community_cards_deck)
    
    player1.meet_other_players([player1, player2])
    player2.meet_other_players([player1, player2])
    
    return {
        "bank": bank,
        "board": board,
        "roads": roads,
        "properties": properties,
        "community_chest_cards": community_chest_cards,
        "players": [player1, player2]
    }

In [132]:
def get_winner(players):
    """
    Determine the winner among players. 
    
    If exactly one player has lost, the other is the winner. 
    If neither has lost by the time we stop, pick the one with the 
    greatest net worth (cash + mortgageable_amount) as the winner.
    Returns:
        winner (Player)
    """
    active_players = [p for p in players if not p.has_lost()]

    if len(active_players) == 1:
        # Exactly one survivor
        return active_players[0]
    else:
        # More than one still in => compare net worth
        # If there's a tie in net worth, we return None
        top_player = None
        top_net_worth = -1
        
        for p in active_players:
            st = p.get_state()
            net_worth = st["cash"] + st["mortgageable_amount"]
            if net_worth > top_net_worth:
                top_player = p
                top_net_worth = net_worth
            elif net_worth == top_net_worth:
                # tie
                top_player = None
        
        return top_player

In [133]:
def players_state(players) -> str:
    """
    Helper function to get a logstring showing all players' states.
    """

    state_obj = []
    for player in players:
        state_obj += [player.get_state()]
    
    return json.dumps(state_obj, indent=2)

### Run a Game

In [134]:
def run_experiment(
    open_ai_key: str,
    raw_log_filepath: str,
    win_rate_filepath: str,
    max_steps_per_game: int = 10,
    num_games: int = 10,
    reflect_before_vectorstore: bool = False,
    primary_model: str = "gpt-4o-mini",
    primary_model_params: dict = {},
    secondary_model: str = "gpt-4o",
    secondary_model_params: dict = {},
    random_seed: int = 42
) -> None:
    """
    Run an experiment with the given parameters, ensuring ALL data are saved.

    Parameters
    ----------
    open_ai_key : str
        The OpenAI API key.
    raw_log_filepath : str
        Base filepath to save outputs from all LLM completions and vectorstore 
        search results for each game. We'll append `_game{i}_timestamp.log`.
    win_rate_filepath : str
        Base filepath to save CSV of game outcomes (two columns: 
        'raw_log_filepath' and 'custom_agent_won'). We'll append `_timestamp.csv`.
    max_steps_per_game : int, optional
        The maximum number of steps per game.
    num_games : int, optional
        The number of games to play to get the average win rate.
    reflect_before_vectorstore : bool, optional
        Whether to reflect and summarize data before adding past context to 
        vectorstore.
    primary_model : str, optional
        Name of the primary “fast” LLM to use (for lawyer chains).
    primary_model_params : dict, optional
        Key-value pairs for the primary LLM (e.g. {"temperature": 0.7}).
    secondary_model : str, optional
        Name of the secondary “slow” LLM to use (for judge chains).
    secondary_model_params : dict, optional
        Key-value pairs for the secondary LLM (e.g. {"temperature": 0.0}).
    """

    # Export model settings to environment variables
    os.environ["OPENAI_API_KEY"] = open_ai_key 
    os.environ["PRIMARY_MODEL_NAME"] = primary_model
    os.environ["SECONDARY_MODEL_NAME"] = secondary_model
    os.environ["REFLECT_BEFORE_VECTORSTORE"] = str(reflect_before_vectorstore)

    if primary_model_params:
        os.environ["PRIMARY_MODEL_PARAMS"] = json.dumps(primary_model_params)
    if secondary_model_params:
        os.environ["SECONDARY_MODEL_PARAMS"] = json.dumps(secondary_model_params)

    # Prepare path for CSV that tracks win/loss for each game
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    final_win_rate_csv = f"logs/fastslow/{random_seed}/{timestamp_str}/{win_rate_filepath}.csv"

    # Ensure the directory exists for the CSV
    os.makedirs(os.path.dirname(final_win_rate_csv), exist_ok=True)
    
    # Set random seed for reproducibility
    random.seed(random_seed)

    # -------------------------------------------------------------------------
    # Run multiple games
    # -------------------------------------------------------------------------
    with open(final_win_rate_csv, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(["raw_log_filepath", "custom_agent_won"])

        for game_index in range(num_games):
            print(f"\nStarting game {game_index}")

            # Archive and reset the vectorstore for each game
            vec = init_vectorstore(timestamp_str, game_index)

            # We'll log everything into a single .log file for this game
            game_log_file = f"logs/fastslow/{random_seed}/{timestamp_str}/{raw_log_filepath}_game{game_index}.log"
            game_tsv_file = f"logs/fastslow/{random_seed}/{timestamp_str}/{raw_log_filepath}_game{game_index}.tsv"
            os.makedirs(os.path.dirname(game_log_file), exist_ok=True)
            os.makedirs(os.path.dirname(game_tsv_file), exist_ok=True)

            os.environ["LOGGER_PATH"] = game_tsv_file
            logger = StructuredLogger(game_tsv_file)
            
            logging.basicConfig(
                filename=game_log_file, level=logging.INFO, force=True,
                format="%(asctime)s - %(levelname)s - %(message)s"
            )
            config = {
                "max_steps_per_game": max_steps_per_game,
                "num_games": num_games,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "primary_model": primary_model,
                "primary_model_params": primary_model_params,
                "secondary_model": secondary_model,
                "secondary_model_params": secondary_model_params,
            }
            logging.info(f"Starting game {game_index} with config: {config}")
            logger.log("game_init", config)
            
            game_data = initialize_game(vec)
            players = game_data["players"]

            # Run up to max_steps_per_game
            step = 0
            while step < max_steps_per_game:
                # Logging progress
                print(step, end=", ")
                if (step + 1) % 30 == 0:
                    print("")
                logging.info("===")
                logging.info(f"Step {step}")
                logging.info("===")
                logger.log("step_num", step)
                logger.log("player_summary", players_state(players))

                # Check if the game already ended
                if any(p.has_lost() for p in players):
                    break

                for p in players:
                    if not p.has_lost():
                        p.play()

                step += 1

            # Determine winner
            winner = get_winner(players)

            # Check if the winner is our custom agent (player1 in this setup)
            custom_agent = players[0]
            custom_agent_won = (winner == custom_agent)

            # Write to CSV
            writer.writerow([game_log_file, str(custom_agent_won)])

    print(f"\nAll {num_games} games complete. Results saved to: {final_win_rate_csv}")

In [135]:
run_experiment(
    open_ai_key=openai_key,
    raw_log_filepath="fastslow_raw",
    win_rate_filepath="fastslow_win_rate",
    max_steps_per_game=200,
    num_games=20,
    reflect_before_vectorstore=True,
    primary_model="gpt-4o-mini",
    primary_model_params={"temperature": 1.0, "max_tokens": 2048},
    secondary_model="gpt-4o",
    secondary_model_params={"temperature": 1.0, "max_tokens": 2048},
    random_seed = 42
)


Starting game 0
0, 1, 

KeyboardInterrupt: 